# Finding Best VQC Model

The below notebook creates a frame work for finding the best model parameters based on results from grid search

In [97]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [98]:
with open ("z_grid_results.txt", "r") as file:
    data_str = file.read()
    
data_list = eval(data_str, {"array": np.array})
print(data_list)

[{'combo_num': 1, 'ansatz': 'EfficientSU2', 'ansatz_reps': 2, 'feature_map': 'ZFeatureMap', 'fm_reps': 2, 'optmizer': 'COBYLA', 'train_time': 1972.9104430675507, 'train_score': 0.5771428571428572, 'test_score': 0.5887445887445888, 'accuracy': 0.5757575757575758, 'p_r_f1_s': (array([0.72807018, 0.42735043]), array([0.55333333, 0.61728395]), array([0.62878788, 0.50505051]), array([150,  81]))}, {'combo_num': 2, 'ansatz': 'EfficientSU2', 'ansatz_reps': 2, 'feature_map': 'ZFeatureMap', 'fm_reps': 2, 'optmizer': 'COBYLA', 'train_time': 1887.9535567760468, 'train_score': 0.6485714285714286, 'test_score': 0.5930735930735931, 'accuracy': 0.5930735930735931, 'p_r_f1_s': (array([0.7       , 0.42857143]), array([0.65333333, 0.48148148]), array([0.67586207, 0.45348837]), array([150,  81]))}, {'combo_num': 3, 'ansatz': 'EfficientSU2', 'ansatz_reps': 2, 'feature_map': 'ZFeatureMap', 'fm_reps': 2, 'optmizer': 'COBYLA', 'train_time': 1922.5225932598114, 'train_score': 0.5671428571428572, 'test_score':

In [99]:
df = pd.DataFrame(data_list)

In [100]:
df[['precision', 'recall', 'f1', 'support']] = pd.DataFrame(df['p_r_f1_s'].tolist(), index=df.index)

In [101]:
df['mean_precision'] = df['precision'].apply(np.mean)
df['mean_recall'] = df['recall'].apply(np.mean)
df['mean_f1'] = df['f1'].apply(np.mean)

In [102]:
df['composite'] = df[['accuracy', 'mean_precision', 'mean_recall', 'mean_f1']].mean(axis=1)

In [103]:
with open("zz_grid_results.txt", "r") as file:
    file_data = file.read()

In [104]:
pattern = (
    r"=====COMBINATION:\s*(\d+)=====\s*"
    r"Ansatz:\s*(.*?)\s*"
    r"Ansatz Reps:\s*(\d+)\s*"
    r"Feature Map:\s*(.*?)\s*"
    r"Feature Map Reps:\s*(\d+)\s*"
    r"Optimizer:\s*(.*?)\s*"
    r"ACCURACY:\s*([\d.]+)\s*"
    r"P R F1 S:\s*\(array\((.*?)\),\s*array\((.*?)\),\s*array\((.*?)\),\s*array\((.*?)\)\)"
)

In [105]:
matches = re.findall(pattern, file_data, re.DOTALL)

In [106]:
data = []
for m in matches:
    combination, ansatz, ansatz_reps, feature_map, fm_reps, optmizer, accuracy, prec, rec, f1, support = m
    entry = {
        "combo_num": int(combination),
        "ansatz": ansatz.strip(),
        "ansatz_reps": int(ansatz_reps),
        "feature_map": feature_map.strip(),
        "fm_reps": int(fm_reps),
        "optmizer": optmizer.strip(),
        "train_time": np.nan,
        "train_score": np.nan,
        "test_score": np.nan,
        "accuracy": float(accuracy),
        "p_r_f1_s": np.nan,
        "precision": np.array(eval(prec)),
        "recall": np.array(eval(rec)),
        "f1": np.array(eval(f1)),
        "support": np.array(eval(support))
    }
    df.loc[len(df)] = entry

In [107]:
df['mean_precision'] = df['precision'].apply(np.mean)
df['mean_recall'] = df['recall'].apply(np.mean)
df['mean_f1'] = df['f1'].apply(np.mean)

In [108]:
df['composite'] = df[['accuracy', 'mean_precision', 'mean_recall', 'mean_f1']].mean(axis=1)

In [109]:
df.nlargest(5, 'accuracy')

,combo_num,ansatz,ansatz_reps,feature_map,fm_reps,optmizer,train_time,train_score,test_score,accuracy,p_r_f1_s,precision,recall,f1,support,mean_precision,mean_recall,mean_f1,composite
14,15,RealAmplitudes,3,ZFeatureMap,2,COBYLA,1586.695409,0.620000,0.649351,0.658009,"([0.76296296, 0.51041667], [0.68666667, 0.6049...","[0.76296296, 0.51041667]","[0.68666667, 0.60493827]","[0.72280702, 0.55367232]","[150, 81]",0.636690,0.645802,0.638240,0.644685
20,21,RealAmplitudes,4,ZFeatureMap,2,COBYLA,1810.431421,0.620000,0.619048,0.636364,"([0.73239437, 0.48314607], [0.69333333, 0.5308...","[0.73239437, 0.48314607]","[0.69333333, 0.5308642]","[0.71232877, 0.50588235]","[150, 81]",0.607770,0.612099,0.609106,0.616335
18,19,EfficientSU2,4,ZFeatureMap,2,COBYLA,2927.570249,0.581429,0.640693,0.627706,"([0.72222222, 0.47126437], [0.69333333, 0.5061...","[0.72222222, 0.47126437]","[0.69333333, 0.50617284]","[0.70748299, 0.48809524]","[150, 81]",0.596743,0.599753,0.597789,0.605498
56,57,EfficientSU2,3,ZFeatureMap,4,COBYLA,2884.950906,0.621429,0.610390,0.623377,"([0.70860927, 0.4625], [0.71333333, 0.45679012...","[0.70860927, 0.4625]","[0.71333333, 0.45679012]","[0.71096346, 0.45962733]","[150, 81]",0.585555,0.585062,0.585295,0.594822
67,68,EfficientSU2,4,ZFeatureMap,4,COBYLA,2882.569408,0.612857,0.610390,0.623377,"([0.72340426, 0.46666667], [0.68, 0.51851852],...","[0.72340426, 0.46666667]","[0.68, 0.51851852]","[0.70103093, 0.49122807]","[150, 81]",0.595035,0.599259,0.596129,0.603450


In [110]:
df.nlargest(5, 'mean_f1')

,combo_num,ansatz,ansatz_reps,feature_map,fm_reps,optmizer,train_time,train_score,test_score,accuracy,p_r_f1_s,precision,recall,f1,support,mean_precision,mean_recall,mean_f1,composite
14,15,RealAmplitudes,3,ZFeatureMap,2,COBYLA,1586.695409,0.620000,0.649351,0.658009,"([0.76296296, 0.51041667], [0.68666667, 0.6049...","[0.76296296, 0.51041667]","[0.68666667, 0.60493827]","[0.72280702, 0.55367232]","[150, 81]",0.636690,0.645802,0.638240,0.644685
20,21,RealAmplitudes,4,ZFeatureMap,2,COBYLA,1810.431421,0.620000,0.619048,0.636364,"([0.73239437, 0.48314607], [0.69333333, 0.5308...","[0.73239437, 0.48314607]","[0.69333333, 0.5308642]","[0.71232877, 0.50588235]","[150, 81]",0.607770,0.612099,0.609106,0.616335
23,24,RealAmplitudes,4,ZFeatureMap,2,COBYLA,1841.917104,0.637143,0.627706,0.614719,"([0.74796748, 0.46296296], [0.61333333, 0.6172...","[0.74796748, 0.46296296]","[0.61333333, 0.61728395]","[0.67399267, 0.52910053]","[150, 81]",0.605465,0.615309,0.601547,0.609260
59,60,EfficientSU2,3,ZFeatureMap,4,COBYLA,2628.430517,0.582857,0.593074,0.606061,"([0.76576577, 0.45833333], [0.56666667, 0.6790...","[0.76576577, 0.45833333]","[0.56666667, 0.67901235]","[0.651341, 0.54726368]","[150, 81]",0.612050,0.622840,0.599302,0.610063
18,19,EfficientSU2,4,ZFeatureMap,2,COBYLA,2927.570249,0.581429,0.640693,0.627706,"([0.72222222, 0.47126437], [0.69333333, 0.5061...","[0.72222222, 0.47126437]","[0.69333333, 0.50617284]","[0.70748299, 0.48809524]","[150, 81]",0.596743,0.599753,0.597789,0.605498


In [111]:
df.nlargest(5, 'composite')

,combo_num,ansatz,ansatz_reps,feature_map,fm_reps,optmizer,train_time,train_score,test_score,accuracy,p_r_f1_s,precision,recall,f1,support,mean_precision,mean_recall,mean_f1,composite
14,15,RealAmplitudes,3,ZFeatureMap,2,COBYLA,1586.695409,0.620000,0.649351,0.658009,"([0.76296296, 0.51041667], [0.68666667, 0.6049...","[0.76296296, 0.51041667]","[0.68666667, 0.60493827]","[0.72280702, 0.55367232]","[150, 81]",0.636690,0.645802,0.638240,0.644685
20,21,RealAmplitudes,4,ZFeatureMap,2,COBYLA,1810.431421,0.620000,0.619048,0.636364,"([0.73239437, 0.48314607], [0.69333333, 0.5308...","[0.73239437, 0.48314607]","[0.69333333, 0.5308642]","[0.71232877, 0.50588235]","[150, 81]",0.607770,0.612099,0.609106,0.616335
59,60,EfficientSU2,3,ZFeatureMap,4,COBYLA,2628.430517,0.582857,0.593074,0.606061,"([0.76576577, 0.45833333], [0.56666667, 0.6790...","[0.76576577, 0.45833333]","[0.56666667, 0.67901235]","[0.651341, 0.54726368]","[150, 81]",0.612050,0.622840,0.599302,0.610063
23,24,RealAmplitudes,4,ZFeatureMap,2,COBYLA,1841.917104,0.637143,0.627706,0.614719,"([0.74796748, 0.46296296], [0.61333333, 0.6172...","[0.74796748, 0.46296296]","[0.61333333, 0.61728395]","[0.67399267, 0.52910053]","[150, 81]",0.605465,0.615309,0.601547,0.609260
18,19,EfficientSU2,4,ZFeatureMap,2,COBYLA,2927.570249,0.581429,0.640693,0.627706,"([0.72222222, 0.47126437], [0.69333333, 0.5061...","[0.72222222, 0.47126437]","[0.69333333, 0.50617284]","[0.70748299, 0.48809524]","[150, 81]",0.596743,0.599753,0.597789,0.605498


In [ ]:
# best row found (combo == 15, w/ z feature map)
# 
df[df["combo_num"] == 15]

,combo_num,ansatz,ansatz_reps,feature_map,fm_reps,optmizer,train_time,train_score,test_score,accuracy,p_r_f1_s,precision,recall,f1,support,mean_precision,mean_recall,mean_f1,composite
14,15,RealAmplitudes,3,ZFeatureMap,2,COBYLA,1586.695409,0.62,0.649351,0.658009,"([0.76296296, 0.51041667], [0.68666667, 0.6049...","[0.76296296, 0.51041667]","[0.68666667, 0.60493827]","[0.72280702, 0.55367232]","[150, 81]",0.636690,0.645802,0.638240,0.644685
86,15,RealAmplitudes,3,ZZFeatureMap,2,COBYLA,NaN,NaN,NaN,0.562771,NaN,"[0.67625899, 0.39130435]","[0.62666667, 0.44444444]","[0.65051903, 0.41618497]","[150, 81]",0.533782,0.535556,0.533352,0.541365
